## Deploying Strands Agent on LSD: Using LangGraph Graph

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### Step 1. Define Strands Agent

In [2]:
from strands import Agent
from strands_tools import file_read, file_write, python_repl, shell, journal

agent = Agent(
        tools=[file_read, file_write, python_repl, shell, journal],
        system_prompt="You are an Expert Software Developer specializing in web frameworks. Your task is to analyze project structures and identify mappings.",
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    )

### Step 2. Use Functional API

The Functional API allows you to integrate and deploy with frameworks other than langchain. 

The Functional API uses two key building blocks:
- `@entrypoint` – Marks a function as the starting point of a workflow, encapsulating logic and managing execution flow, including handling long-running tasks and interrupts.
- `@task` – Represents a discrete unit of work, such as an API call or data processing step, that can be executed asynchronously within an entrypoint. Tasks return a future-like object that can be awaited or resolved synchronously.


Additionally, with Functional API, you have the freedom to leverage LangGraph’s other key features — persistence, memory, human-in-the-loop, and streaming — to other frameworks with minimal changes to your existing code. 

In [3]:
from strands.types.content import Message

from langgraph.func import entrypoint, task
import operator

@task
def invoke_strands(messages: list[Message]):
    # run the agent with existing messages; can invoke with the final message with messages[-1]
    result = agent(messages) 
    # return the resulting message 
    return [result.message]



@entrypoint()
def workflow(messages: list[Message], previous: list[Message]):
    messages = operator.add(previous or [], messages)
    response = invoke_strands(messages).result()
    return entrypoint.final(value=response, save=operator.add(messages, response))

### Step 3. Set up Tracing with OpenTelemetry

In [4]:
from strands.telemetry import StrandsTelemetry

strands_telemetry = StrandsTelemetry()
strands_telemetry.setup_otlp_exporter()
strands_telemetry.setup_meter()

Example invocation & [trace](https://smith.langchain.com/public/17bdd4f5-81cd-48f4-8e28-d8a5b3e88d38/r) from invocation

In [ ]:
# Example invocation 

user_message: Message = [{
    "role": "user",
    "content": [
        {"text": "Do a short review of /Users/qiaocatherine/Documents/Fortune1000_SFDC_First20.csv. Focus on key functionality."}
    ]
}]

for chunk in workflow.stream(user_message):
    print(chunk)

### Step 4. Deploy

Now you can deploy on LangSmith Deployments. Make sure you have the following 
- Dependency file (e.g., `requirements.txt` or `pyproject.toml` 
- Configuration file (`langgraph.json`)
- Relevant environment variables

Check out [here](https://docs.langchain.com/langsmith/setup-app-requirements-txt) for detailed instructions on deploying with dependency or configuration files

For a strands agent, an example list of relevant secrets (see `.env.example`) include: 

`LANGSMITH_TRACING`=false (We will turn of LangSmith default tracing, as we want to only trace with OpenTelemetry)


AWS credentials: 

`AWS_ACCESS_KEY_ID`="your-aws-access-key-id"

`AWS_SECRET_ACCESS_KEY`="your-aws-secret-access-key"

`AWS_REGION`="your-aws-region"


Tracing: 

`OTEL_EXPORTER_OTLP_ENDPOINT` = "https://api.smith.langchain.com/otel/" 

(For self hosting, this would look like your own domain: OTEL_EXPORTER_OTLP_ENDPOINT = "<self-hosted-domain>/api/v1/otel")

`OTEL_EXPORTER_OTLP_HEADERS` = "x-api-key=your-langsmith-api-key,Langsmith-Project=your-tracing-project-name"

`BG_JOB_ISOLATED_LOOPS`=true (execute background runs in an isolated event loop separate from the serving API event loop; Strand's OTel tracing contains synchronous code)